In [1]:
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11.0.11"
os.environ["SPARK_HOME"] = f"C:\spark\spark-3.1.2-bin-hadoop3.2"
import findspark
import pyspark
from pyspark.sql import SparkSession


In [2]:
findspark.init()
spark = SparkSession.builder.appName("emissionsdataframe").getOrCreate()

In [13]:
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, update, Table
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
# from sql_query_to_csv import sql_query_to_csv
rds_string = 'postgres:postgres@emissions.cgiuxtnnybgt.us-west-1.rds.amazonaws.com:5432/emissions_db'

In [8]:
engine = create_engine(f'postgresql://{rds_string}')
metadata = MetaData(engine)
gdp_data = Table('gdp_data', metadata, autoload=True, autoload_with=engine)


In [17]:
query = session.query(gdp_data).all()


In [23]:
import pandas as pd
conn = engine.connect()

In [26]:
gdp_df = pd.read_sql_query('select * from gdp_data', con=conn)

In [27]:
emmissions_df = pd.read_sql_query('select * from global_emissions', con=conn)

In [35]:
india_df = pd.read_sql_query('select * from india_export_data', con=conn)

In [33]:
trade_df = pd.read_sql_query('select * from global_trade', con=conn)

In [67]:
schem = gdp_df.columns
schema_list = []
for i in schem:
    schema_list.append(i)

In [45]:
gdp_csv = gdp_df.to_csv(index=False)

In [68]:
sparkDF=spark.createDataFrame(gdp_df, verifySchema=True, schema=schema_list)

In [48]:
from pyspark.sql.types import StructType, StructField, StringType
schema = StructType([StructField("foo", StringType(), True)])
# df = spark.createDataFrame([[None]], schema=schema)

In [49]:
df = spark.createDataFrame([[None]], schema=schema)

In [50]:
df.show()

+----+
| foo|
+----+
|null|
+----+



In [55]:
gdp_df = gdp_df.fillna(0)

In [70]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")
sparkDF=spark.createDataFrame(gdp_df) 
sparkDF.printSchema()
sparkDF.show()

C:\Users\Starkiller\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\sql\pandas\conversion.py:289: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


root
 |-- Country : string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- 1990: double (nullable = true)
 |-- 1991: double (nullable = true)
 |-- 1992: double (nullable = true)
 |-- 1993: double (nullable = true)
 |-- 1994: double (nullable = true)
 |-- 1995: double (nullable = true)
 |-- 1996: double (nullable = true)
 |-- 1997: double (nullable = true)
 |-- 1998: double (nullable = true)
 |-- 1999: double (nullable = true)
 |-- 2000: double (nullable = true)
 |-- 2001: double (nullable = true)
 |-- 2002: double (nullable = true)
 |-- 2003: double (nullable = true)
 |-- 2004: double (nullable = true)
 |-- 2005: double (nullable = true)
 |-- 2006: double (nullable = true)
 |-- 2007: double (nullable = true)
 |-- 2008: double (nullable = true)
 |-- 2009: double (nullable = true)
 |-- 2010: double (nullable = true)
 |-- 2011: double (nullable = true)
 |-- 2012: double (nullable = true)
 |-- 2013: double (nullable = true)
 |-- 2014: double (nullable = true)
 |-- 2015: 

In [82]:
sparkDF.filter(sparkDF["Country "] == "Afghanistan").select(['Country ']).show()

+-----------+
|   Country |
+-----------+
|Afghanistan|
+-----------+



In [83]:
sparkDF.select(['Country ']).show()

+--------------------+
|            Country |
+--------------------+
|               Aruba|
|         Afghanistan|
|              Angola|
|             Albania|
|          Arab World|
|United Arab Emirates|
|           Argentina|
|             Armenia|
| Antigua and Barbuda|
|           Australia|
|             Austria|
|          Azerbaijan|
|             Burundi|
|             Belgium|
|               Benin|
|        Burkina Faso|
|          Bangladesh|
|            Bulgaria|
|             Bahrain|
|        Bahamas, The|
+--------------------+
only showing top 20 rows



In [80]:
sparkDF.columns

['Country ',
 'Country Code',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']